## Deliverable 2. Create a Customer Travel Destinations Map.

In [67]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [68]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,60.01,56,35,8.01,light rain
1,1,Mishkino,RU,55.3389,63.9175,-1.95,90,0,12.06,clear sky
2,2,Gat,IL,31.6100,34.7642,55.00,100,83,5.99,moderate rain
3,3,Chuy,UY,-33.6971,-53.4616,71.42,60,75,9.08,broken clouds
4,4,Port Lincoln,AU,-34.7333,135.8667,60.39,77,100,11.86,overcast clouds


In [69]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 90


In [70]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Sacama,EC,-3.9000,-79.2667,77.00,61,75,12.66,moderate rain
6,6,Ouegoa,NC,-20.3500,164.4333,83.71,75,92,10.07,light rain
7,7,Kapaa,US,22.0752,-159.3190,80.60,69,20,19.57,few clouds
10,10,Chilca,PE,-12.5211,-76.7372,75.20,64,0,17.27,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,78.51,77,50,14.81,scattered clouds
18,18,Porto Novo,BJ,6.4965,2.6036,82.40,83,20,9.22,few clouds
22,22,Fare,PF,-16.7000,-151.0167,81.61,74,0,13.02,clear sky
23,23,Lompoc,US,34.6391,-120.4579,82.40,18,0,12.66,clear sky
24,24,Meulaboh,ID,4.1363,96.1285,76.30,84,97,4.12,overcast clouds
39,39,Cabo San Lucas,MX,22.8909,-109.9124,78.80,27,1,5.75,clear sky


In [71]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                226
City                   226
Country                225
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [72]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = "NaN"
preferred_cities_df.replace("", nan_value, inplace=True)
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

C:\Users\mspir\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4385: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\mspir\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [73]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_df = preferred_cities_df
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Sacama,EC,77.00,moderate rain,-3.9000,-79.2667,
6,Ouegoa,NC,83.71,light rain,-20.3500,164.4333,
7,Kapaa,US,80.60,few clouds,22.0752,-159.3190,
10,Chilca,PE,75.20,clear sky,-12.5211,-76.7372,
16,Atuona,PF,78.51,scattered clouds,-9.8000,-139.0333,
18,Porto Novo,BJ,82.40,few clouds,6.4965,2.6036,
22,Fare,PF,81.61,clear sky,-16.7000,-151.0167,
23,Lompoc,US,82.40,clear sky,34.6391,-120.4579,
24,Meulaboh,ID,76.30,overcast clouds,4.1363,96.1285,
39,Cabo San Lucas,MX,78.80,clear sky,22.8909,-109.9124,


In [74]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [75]:
#Inspect the new DataFrame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Sacama,EC,77.00,moderate rain,-3.9000,-79.2667,Hacienda Gonzabal
6,Ouegoa,NC,83.71,light rain,-20.3500,164.4333,
7,Kapaa,US,80.60,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
10,Chilca,PE,75.20,clear sky,-12.5211,-76.7372,Playa Punta Yaya - Las Salinas
16,Atuona,PF,78.51,scattered clouds,-9.8000,-139.0333,Villa Enata
18,Porto Novo,BJ,82.40,few clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
22,Fare,PF,81.61,clear sky,-16.7000,-151.0167,Maitai Lapita Village Huahine
23,Lompoc,US,82.40,clear sky,34.6391,-120.4579,Holiday Inn Express Lompoc
24,Meulaboh,ID,76.30,overcast clouds,4.1363,96.1285,Kost Pria
39,Cabo San Lucas,MX,78.80,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [77]:
print(hotel_df.loc[hotel_df["Hotel Name"]=="",:])

          City Country  Max Temp Current Description      Lat       Lng  \
6       Ouegoa      NC     83.71          light rain -20.3500  164.4333   
257      Kieta      PG     82.67          light rain  -6.2167  155.6333   
328     Aitape      PG     81.19       broken clouds  -3.1374  142.3475   
410   Marovoay      MG     82.40          light rain -16.1000   46.6333   
438  Sembakung      ID     75.20          few clouds   3.8167  117.2500   
497     Beloha      MG     74.25     overcast clouds -25.1667   45.0500   
583   Mabaruma      GY     74.91     overcast clouds   8.2000  -59.7833   
609    Madimba      TZ     80.60          few clouds -10.4000   40.3333   
640     Tucuma      BR     77.79           clear sky  -0.1833  -50.0500   

    Hotel Name  
6               
257             
328             
410             
438             
497             
583             
609             
640             


In [79]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Sacama,EC,77.00,moderate rain,-3.9000,-79.2667,Hacienda Gonzabal
7,Kapaa,US,80.60,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
10,Chilca,PE,75.20,clear sky,-12.5211,-76.7372,Playa Punta Yaya - Las Salinas
16,Atuona,PF,78.51,scattered clouds,-9.8000,-139.0333,Villa Enata
18,Porto Novo,BJ,82.40,few clouds,6.4965,2.6036,Tour Eiffel Hotel Benin
22,Fare,PF,81.61,clear sky,-16.7000,-151.0167,Maitai Lapita Village Huahine
23,Lompoc,US,82.40,clear sky,34.6391,-120.4579,Holiday Inn Express Lompoc
24,Meulaboh,ID,76.30,overcast clouds,4.1363,96.1285,Kost Pria
39,Cabo San Lucas,MX,78.80,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
41,Rikitea,PF,78.96,clear sky,-23.1203,-134.9692,Pension Maro'i


In [80]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [82]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [83]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))